# MHC Class I-Peptides Binding Affinity Prediction

In this tutorial, we will build a convolutional network + LSTM to do MHC clss I binding affinity prediction from a Benchmark data set from [Kim _et al_. 2014](http://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-15-241)

## Prerequisite

### Define paths

In [1]:
# The path corresponds to where you clone the bioitworld2017 repository
PRJ = "/workspace/bioitworld2017"

### Import modules

In [2]:
from __future__ import print_function
from Bio.Alphabet import IUPAC
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import scipy.stats as stats
import sys
from sklearn.metrics import roc_auc_score

%matplotlib inline

We will use the [Keras](http://keras.io) library to build the deep learning model. It is a very straightforward wrapper around the popular tensor-based library [Theano](http://deeplearning.net/software/theano/introduction.html) and Google's [TensorFlow](https://www.tensorflow.org/). The user just need to connect the layers, the library will build the low-level parameters and operations by calling the backend libraries. Eventually the model is translated into C++ code for speed.

In [21]:
import keras
from keras.layers import (
    Activation, BatchNormalization, Conv1D, MaxPooling1D,
    Dense, Dropout, Embedding, LSTM, 
)
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.preprocessing import sequence
from keras.regularizers import l1_l2

## Read data set

Most time we are spending when building a machine learning model is in two things:

  1. Cleaning and consolidating data set
  2. Tuning parameters
  
I have previously re-formatting the MHC class I benchmark data set into training-ready format. Everything is wrapped within a [`pickle`](https://docs.python.org/2/library/pickle.html) file. 

In [11]:
df, X, y, meta, sample_weights,AA_MAP = pickle.load(
    open(os.path.join(PRJ, "data/mhci-v0.2.pkl"), "rb")
)

The pickel file contains following information:

#### Raw data frame: `df`

The original data frame read from the data set.

In [4]:
df.sample(10)

,species,mhc,peptide_length,cv,sequence,inequality,meas
146293,human,HLA-B*46:01,9,TBD,TTYVYTLPV,>,20000.000000
46867,human,HLA-A*02:19,9,TBD,MLKLRVDVF,>,20000.000000
178717,chimpanzee,Patr-B*0101,10,TBD,VLKPGMVVTF,>,69811.320755
40889,human,HLA-A*02:06,10,TBD,DLAIKQYGDI,=,46835.443038
70389,human,HLA-A*26:01,9,TBD,KLYERNTAF,>,20000.000000
109812,human,HLA-B*07:02,10,TBD,VPSIKSGNDI,=,1377.283622
152591,human,HLA-B*54:01,9,TBD,VPSHISSLI,=,7455.752022
170504,macaque,Mamu-B*03,11,TBD,IRQVLFLEKIE,=,7074.450208
7527,mouse,H-2-Kb,9,TBD,VQYSNYSFL,=,1.080000
89889,human,HLA-A*33:01,9,TBD,QNPTMLYNK,=,23442.899447


#### Data matrices: `X` and `y`

The transformed `numpy.ndarray` that can directly fed to the model

  * `X` : the encoded amino acid sequence
  * `y` : a vector of size (`num_samples`) with the log10 binding affinity for each peptide to the HLA subtype

In [7]:
print("Shape of X: {}".format(X.shape))
print("Shape of y: {}".format(y.shape))

Shape of X: (151933,)
Shape of y: (151933,)


#### Meta data: `meta`

A data frame that connects the matrices back to the annotation, including the randomly split training and test set label (20% of test).

In [8]:
meta.sample(10)

,species,mhc,peptide_length,cv,sequence,inequality,meas,log10_meas,peptide_code,sample_weights,set
74528,human,HLA-A*31:01,9,TBD,RQLESRLGY,=,287.937493,2.459298,"[14, 13, 9, 3, 15, 14, 9, 5, 19]",1.0,train
84137,human,HLA-A*68:01,9,TBD,TLNHVLALK,=,4.168094,0.619938,"[16, 9, 11, 6, 17, 9, 0, 9, 8]",1.0,train
19107,human,HLA-A*02:02,10,TBD,ALMIAAQVVV,=,50.395007,1.702388,"[0, 9, 10, 7, 0, 0, 13, 17, 17, 17]",1.0,train
112831,human,HLA-B*18:01,9,TBD,ADMSKLISL,>,77500.000000,4.889302,"[0, 2, 10, 15, 8, 9, 7, 15, 9]",0.5,train
90605,human,HLA-A*68:02,10,TBD,IVFGIYKDNL,=,628.070550,2.798008,"[7, 17, 4, 5, 7, 19, 8, 2, 11, 9]",1.0,train
91965,human,HLA-A*69:01,9,TBD,ARIDARIDF,>,20000.000000,4.301030,"[0, 14, 7, 2, 0, 14, 7, 2, 4]",0.5,train
109569,human,HLA-B*15:03,9,TBD,FMNKHILSY,<,1.000000,0.000000,"[4, 10, 11, 8, 6, 7, 9, 15, 19]",0.5,train
42449,human,HLA-A*03:01,9,TBD,VSDLYTSMR,=,2488.845673,3.395998,"[17, 15, 2, 9, 19, 16, 15, 10, 14]",1.0,train
86662,human,HLA-A*68:02,9,TBD,DEFLKVPEW,>,20000.000000,4.301030,"[2, 3, 4, 9, 8, 17, 12, 3, 18]",0.5,train
35323,human,HLA-A*02:16,9,TBD,HQIWLALRY,>,20000.000000,4.301030,"[6, 13, 7, 18, 9, 0, 9, 14, 19]",0.5,train


#### Amino acid code book: `AA_MAP`

The mapping between the code in `X` and the amino acid.

In [9]:
IUPAC.extended_protein.letters

'ACDEFGHIKLMNPQRSTVWYBXZJUO'

In [12]:
AA_MAP

{'A': 0,
 'B': 20,
 'C': 1,
 'D': 2,
 'E': 3,
 'F': 4,
 'G': 5,
 'H': 6,
 'I': 7,
 'J': 23,
 'K': 8,
 'L': 9,
 'M': 10,
 'N': 11,
 'O': 25,
 'P': 12,
 'Q': 13,
 'R': 14,
 'S': 15,
 'T': 16,
 'U': 24,
 'V': 17,
 'W': 18,
 'X': 21,
 'Y': 19,
 'Z': 22}

#### Sample weights: `sample_weights`

From the `meta` data we can see there are some measurements with inequalities, _e.g._ > 20,000. If our objective is to get the minimum error from the predicted binding affinities, we have to down-weight these samples. For simplicity, we down-weighted these samples' weights by half, as you also see in `meta`.

In [13]:
sample_weights

array([ 1. ,  0.5,  0.5, ...,  1. ,  1. ,  1. ])

### Subsetting and splitting the data set

Let's first check how many samples we have for each HLA subtype:

In [14]:
meta.groupby("mhc").size()

mhc
HLA-A*01:01     4558
HLA-A*02:01    11920
HLA-A*02:02     4155
HLA-A*02:03     6302
HLA-A*02:04        4
HLA-A*02:05       75
HLA-A*02:06     5627
HLA-A*02:07       80
HLA-A*02:10       18
HLA-A*02:11     1085
HLA-A*02:12     1183
HLA-A*02:16      920
HLA-A*02:17      346
HLA-A*02:19     1245
HLA-A*02:50      135
HLA-A*03:01     7089
HLA-A*03:02       26
HLA-A*03:19       30
HLA-A*11:01     6255
HLA-A*11:02       14
HLA-A*23:01     2508
HLA-A*24:02     3230
HLA-A*24:03     1176
HLA-A*25:01      960
HLA-A*26:01     4326
HLA-A*26:02      643
HLA-A*26:03      536
HLA-A*29:02     2658
HLA-A*30:01     2778
HLA-A*30:02     1927
               ...  
HLA-B*52:01       15
HLA-B*53:01     1603
HLA-B*54:01     1203
HLA-B*57:01     2776
HLA-B*57:02       18
HLA-B*57:03       34
HLA-B*58:01     3121
HLA-B*58:02       51
HLA-B*73:01      122
HLA-B*81:01       26
HLA-B*83:01      338
HLA-B27            2
HLA-B44            5
HLA-B51            3
HLA-B60            5
HLA-B7            67
HLA-B8   

For a lot of HLA types, we don't have a lot of samples. The current state-of-art approaches such as [NetMHC](http://www.cbs.dtu.dk/services/NetMHC/) train one model for one HLA subtype. Here let's focus on the largest one: HLA-A0201. We also only use peptides that's shorter than 15 amino acids.

We also want to split the data set into training and testing based on the index in the metadata. The testing set will not be used in learning weights in the network in any way. We will only use it to evaluate if our model is overfitting and select a stopping point for training.

One other thing: we have to reshape the vector of `X_train` and `X_test` into a matrix. We can use the `keras.preprocessing.sequence.pad_sequences` to pad the shorter sequences with an extra code (in our case, 26, given that the original coding is from 0-25).

Note that we convert the matrix to float 32 for the benefit of running it on GPU.

In [17]:
max_len = 14
# add one padding feature
max_features = len(IUPAC.extended_protein.letters) + 1 

train_idx = meta[(meta["set"] == "train") & 
                 (meta["mhc"] == "HLA-A*02:01") & 
                 (meta["peptide_length"] <= max_len)].index.values
test_idx = meta[(meta["set"] == "test") &
                (meta["mhc"] == "HLA-A*02:01") & 
                (meta["peptide_length"] <= max_len)].index.values

X_train = X[train_idx]
y_train = y[train_idx]
X_test = X[test_idx]
y_test = y[test_idx]

X_train = sequence.pad_sequences(X_train, maxlen=max_len, 
                                 value=max_features-1)
X_test = sequence.pad_sequences(X_test, maxlen=max_len, 
                                value=max_features-1)

w_train = sample_weights[train_idx].astype(np.float32)
w_test = sample_weights[test_idx].astype(np.float32)

## Build a model

Now let's build a deep learning model!

### Set model parameters

Let's first define some parameters

In [18]:
batch_size = 128 
nb_epoch = 10 # only train for 10 iteration

embedding_size = 32
num_conv_filters = 32 # number of convolutional filters
conv_filter_size = 3 # size of the convolutional filters
num_lstm_units = 64 # number of LSTM units

In [23]:
model = Sequential()

# 4 layers of conv
model.add(Embedding(max_features, embedding_size, input_length=max_len))
model.add(Dropout(0.25))
model.add(Conv1D(filters=num_conv_filters,
                 kernel_size=conv_filter_size))
model.add(BatchNormalization(axis=1))
model.add(Activation("relu"))
model.add(MaxPooling1D(2))

model.add(LSTM(num_lstm_units))
model.add(Dense(1, kernel_regularizer=l1_l2(l1=1e-2, l2=1e-2),
                activation="relu"))

You can access the number of parameters and the shape of output in each layer by the `.summary()` function.

In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 14, 32)            864       
_________________________________________________________________
dropout_4 (Dropout)          (None, 14, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 12, 32)            3104      
_________________________________________________________________
batch_normalization_3 (Batch (None, 12, 32)            48        
_________________________________________________________________
activation_2 (Activation)    (None, 12, 32)            0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 6, 32)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                24832     
__________

### Embedding layers

The embedding layers are often used in NLP (such as this [sentimental analysis from imdb](https://github.com/fchollet/keras/blob/master/examples/imdb_lstm.py)). It turns the indexed input, in our case the encoded amino acid sequences, into a dense vector of fixed size, _e.g._ [4, 20] -> [[0.25, 0.1], [0.6, -0.2]]

The parameters of the embedding layers try to capture the relationships between the encoded elements, for example, the similarities between amino acids or the similar meanings across words. Elements with high similarities will have similar vector representations. 

The number of parameters of an embedding layer is the `dimension_of_codebook` multiplied by the `number of embedding units`. In our case, it's 

$$ 26 \text{ alphabets in the codebook} \times 32 \text{ embedding units} = 832 \text{ parameters}$$

### Dropout layer

Large number of parameters tends to overfit the training data, the number of parameters in deep neural network is gigantic. One simple technique that is offen used to prevent overfitting is to insert a [dropout](http://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf) layer in between fully connected layers (_e.g._ LSTM). A dropout layer randomly set a fraction `p` of input units to 0 at each update during training time: 

![](../figures/dropout.png)

### Convolutional layers

The convolutional layer has a sparse connection to the adjecent units:

![](http://deeplearning.net/tutorial/_images/conv_1D_nn.png)

In the figure above, lines with same color correspond to the same weight. 

For activation, the convolution layers use Rectified Linear Unit (ReLU), which is shown to have [better convergence](http://www.cs.toronto.edu/~fritz/absps/imagenet.pdf) than `tanh` function.

![](http://cs231n.github.io/assets/nn1/relu.jpeg)

You can see the first convolutional layer contains 

$$  32 \text{ embedding units from last layer} \times 32 \text{ convolutional units} \times 3 \text{ (convolutional filter size)} + 32 \text{ bias} = 3104 \text{ parameters}$$

### Batch normalization layer

According to [this paper](https://arxiv.org/pdf/1502.03167v3.pdf) from Google, a batch normalization layer can reduce internal covariate shift between layers and thus reduce the convergence time and might eliminatethe need for Dropout. Each unit in batch normailzation calculates the mean and standard deviation in the mini batch and shift the input by

$$ y = \gamma x + \beta$$

Therefore, the unmber of parameter is $2 \times 14 = 28$ in the first BatchNormalization layer.

### Max pooling layer

A max pooling layer parttions input into non-overlapping rectangles and output the maximum in each subregion. It reduces the data size and thus speed up the computation. Usually the pooling size is 2, too much pooling may result in loss of too much information. 

### LSTM layer

Long short-term memory (LSTM) layer is one of the most widely used recurrent neural network (RNN) layer. The units in the recurrent neural network forms directed cycles which allow them to exhibit temporal behavior. Therefore we often see the application of RNNs in sequential data such as [handwriting recognition](https://arxiv.org/pdf/1312.4569.pdf), [speech recognition](https://www.microsoft.com/en-us/research/publication/lstm-time-and-frequency-recurrence-for-automatic-speech-recognition/), or [sentimental analysis](http://deeplearning.net/tutorial/lstm.html).

A LSTM unit is different from other RNN unit in the sense that it has a forget gate:

![](http://deeplearning.net/tutorial/_images/lstm_memorycell.png)

For each of the unit at time $t$, it involves the following calculation:

  * Input gate 
      $$i_t = \sigma(W_ix_t + U_i h_{t-1} + b_i)$$
  
  * Candidate value for the state of memory cell 
      $$\tilde{C}_t = tanh(W_cx_t + U_ch_{t-1} + b_c)$$
  
  * Activation of forget cell
      $$f_t = \sigma(W_fx_t + U_fh_{t-1} + b_f)$$
  
  * The state of memory cell 
      $$C_t = i_t \tilde{C}_t + f_t C_{t-1}$$
  
  * Output gate
      $$o_t = \sigma(W_ox_t + U_o h_{t-1} + b_o)$$
  
  * Output 
      $$h_t = o_t tanh(C_t)$$

The above calculations involve the following parameters:
  * $W$ : weight vector of `input_size`
  * $U$ : weight vector of `output_size`
  * $b$ : a scalar of bias

Therefore, for instance, the LSTM layer involves

$$(64 \text{ units of LSTM layer} + 32 \text{ units of previous layer} + 1 \text{ bias}) \times 64 \text{ units} \times 4 (\text{input gate, candidate state, forget, output gate}) = 24832 \text{ parameters} $$

## Configure model for training

With the model structure in place, we will configure the training methods for the model.

In [25]:
model.compile(loss="mean_absolute_error",
              optimizer="rmsprop")

### Choose optimizer

Optimizers are basically a gradient descent algorithm used for optimizing the weights during each update. Here we will use the `RMSProp` optimizer as it is suggested to be [a good choice for recurrent neural networks](https://keras.io/optimizers/#rmsprop). A list of other optimizers and their formulation can be found in [this](http://sebastianruder.com/optimizing-gradient-descent/index.html)  great blog post. 

### `compile`

Although the name might be misleading, this function does not really start compiling the model into C++ code, instead just setting more training behavior. Here we use the categorical [crossentropy](https://en.wikipedia.org/wiki/Cross_entropy) as the target value to optimize. In each iteration (or as the deep learning guys like to call it, the `epoch`) we will also output the prediction accuracy as output. 

## Finally ... we start training

And now we can train

In [26]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    validation_data=(X_test, y_test),
                    sample_weight=w_train,
                    shuffle=True)


Train on 9524 samples, validate on 2382 samples
Epoch 1/10
9524/9524 [==============================] - 6s - loss: 1.1099 - val_loss: 2.9796
Epoch 2/10
9524/9524 [==============================] - 6s - loss: 0.9123 - val_loss: 2.9498
Epoch 3/10
9524/9524 [==============================] - 5s - loss: 0.8334 - val_loss: 2.7713
Epoch 4/10
9524/9524 [==============================] - 5s - loss: 0.7765 - val_loss: 2.6282
Epoch 5/10
9524/9524 [==============================] - 5s - loss: 0.7463 - val_loss: 2.4708
Epoch 6/10
9524/9524 [==============================] - 5s - loss: 0.7132 - val_loss: 1.9644
Epoch 7/10
9524/9524 [==============================] - 5s - loss: 0.6953 - val_loss: 1.4714
Epoch 8/10
9524/9524 [==============================] - 5s - loss: 0.6764 - val_loss: 1.2187
Epoch 9/10
9524/9524 [==============================] - 5s - loss: 0.6594 - val_loss: 0.8720
Epoch 10/10
9524/9524 [==============================] - 5s - loss: 0.6512 - val_loss: 0.8044


We can make predictions for new data by doing:

In [27]:
y_test_pred = model.predict(X_test)

There are two ways of evaluating the performance of the model

1. A general practice in terms of binding affinity is to use IC50 < 500 as high affinity. We can use this threshold to calculate the accuracy.
2. We can also calculate the Spearman correlation between the observed affinity with the predicted one.

In [28]:
auc = roc_auc_score(y_test > np.log10(500), y_test_pred)
spearman = stats.spearmanr(y_test, y_test_pred)

print("AUROC: {:.4f}".format(auc))
print("SpearmanR: {:.4f}".format(spearman.correlation))

AUROC: 0.9386
SpearmanR: 0.8055


## More

For other techniques for training model you may go through the following links:

* [IEDB automated server benchmark](http://tools.iedb.org/auto_bench/mhci/weekly/) and [the paper](https://www.ncbi.nlm.nih.gov/pubmed/25717196)
* [`mhcflurry`](https://github.com/hammerlab/mhcflurry) is an open-source project by [Jeff Hammerbacher's lab](http://www.hammerlab.org/). It's a great source to learn about building deep learning models for MHC binding prediction, but the code is not very straitforward ...


* Callback functions: [`ModelCheckpoint`](https://keras.io/callbacks/#modelcheckpoint) or [`EarlyStopping`](https://keras.io/callbacks/#earlystopping) are quite useful
* Hyperparameter search using `sklearn`'s [`GridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) function. An example can be found in `kera`'s [example script](https://github.com/fchollet/keras/blob/master/examples/mnist_sklearn_wrapper.py).
* Parallelization: [`mxnet`](https://github.com/dmlc/mxnet) or [Spark](https://databricks.com/blog/2016/01/25/deep-learning-with-apache-spark-and-tensorflow.html)
* An nice overview on available deep learning library out there (more focused on python): [My Top 9 Favorite Python Deep Learning Libraries](http://www.pyimagesearch.com/2016/06/27/my-top-9-favorite-python-deep-learning-libraries/)